***GENERATED CODE FOR zillowregression PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
from hdfs3 import HDFileSystem
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class HDFSConnector:

    def fetch(spark, config):
        ################### INPUT HADOOP HOST PORT TO CONNECT WITH ###############################
        hdfs = HDFileSystem(
            host=eval(config)['host'], port=eval(config)['port'])
        with hdfs.open(eval(config)['url']) as f:
            df = pd.read_csv(f, error_bad_lines=False)
        df = spark.createDataFrame(dfPd)
        display(df.limit(2).toPandas())
        return df

    def put(df, spark, config):
        return df.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                              delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'City', 'transformation_label': 'String Indexer'}], 'feature': 'City', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': 'Aberdeen', 'max': 'Yuba City', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'City'}, {'feature_label': 'City', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('City')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Metro', 'transformation_label': 'String Indexer'}], 'feature': 'Metro', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Aberdeen', 'max': 'Yuba City', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Metro'}, {'feature_label': 'Metro', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Metro')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'County', 'transformation_label': 'String Indexer'}], 'feature': 'County', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Aiken', 'max': 'York', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'County'}, {'feature_label': 'County', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('County')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'State', 'transformation_label': 'String Indexer'}], 'feature': 'State', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'AL', 'max': 'WV', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'State'}, {'feature_label': 'State', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('State')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from sklearn.model_selection import train_test_split
from tpot import TPOTRegressor
import pyspark


def functionRegression(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTRegressor(verbosity=3, generations=10, max_time_mins=5,
                              n_jobs=-1, random_state=25, population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Error rate of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
############## CREATE SPARK SESSION ############################ ENTER YOUR SPARK MASTER IP AND PORT TO CONNECT TO SERVER ################from pyspark.sql import SparkSession
spark = SparkSession.builder.master('spark://0.0.0.0:0000').getOrCreate()
#%run zillowregressionHooks.ipynb
try:
	#sourcePreExecutionHook()

	exportmergedunion = HDFSConnector.fetch(spark, "{'url': '/FileStore/platform/uploadedSourceFiles/1626397982008_export_merged_union.csv', 'filename': '1626397982008_export_merged_union.csv', 'delimiter': ',', 'file_type': 'Delimeted', 'dbfs_token': '', 'dbfs_domain': '', 'FilePath': '/Real Estate Models /Zillow Rent Index - Time Series/NewZillow/export_merged_union.csv', 'viewFileName': 'export_merged_union.csv', 'is_header': 'Use Header Line', 'server_url': '/numtraPlatform/NumtraPlatformV2/uploads/platform/', 'results_url': 'http://3.140.115.35:4040/api/read/hdfs'}")

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run zillowregressionHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(exportmergedunion,json.dumps( {"FE": [{"transformationsData": [{"transformation_label": "novalue"}], "feature": "City Code", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "83575.88", "stddev": "124244.91", "min": "3344", "max": "399305", "missing": "0"}, "updatedLabel": "City Code"}, {"transformationsData": [{"feature_label": "City", "transformation_label": "String Indexer"}], "feature": "City", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Aberdeen", "max": "Yuba City", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "City"}, {"transformationsData": [{"feature_label": "Metro", "transformation_label": "String Indexer"}], "feature": "Metro", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Aberdeen", "max": "Yuba City", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Metro"}, {"transformationsData": [{"feature_label": "County", "transformation_label": "String Indexer"}], "feature": "County", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Aiken", "max": "York", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "County"}, {"transformationsData": [{"feature_label": "State", "transformation_label": "String Indexer"}], "feature": "State", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AL", "max": "WV", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "State"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Population Rank_x", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "6037.93", "stddev": "3713.81", "min": "8.0", "max": "13086.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Population Rank_x"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "November 2010", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1395.11", "stddev": "668.04", "min": "624.0", "max": "7106.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "November 2010"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "December 2010", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1401.04", "stddev": "671.28", "min": "618.0", "max": "7184.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "December 2010"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "January 2011", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1417.33", "stddev": "703.0", "min": "615.0", "max": "7261.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "January 2011"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "February 2011", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1415.22", "stddev": "703.56", "min": "608.0", "max": "7318.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "February 2011"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "March 2011", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1411.9", "stddev": "703.17", "min": "601.0", "max": "7362.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "March 2011"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "April 2011", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1409.55", "stddev": "703.43", "min": "589.0", "max": "7372.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "April 2011"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "May 2011", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1405.66", "stddev": "701.17", "min": "583.0", "max": "7345.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "May 2011"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "June 2011", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1407.76", "stddev": "699.78", "min": "578.0", "max": "7317.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "June 2011"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "July 2011", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1406.29", "stddev": "697.15", "min": "584.0", "max": "7316.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "July 2011"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "August 2011", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1403.88", "stddev": "696.24", "min": "593.0", "max": "7340.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "August 2011"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "September 2011", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1401.14", "stddev": "696.5", "min": "600.0", "max": "7386.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "September 2011"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "October 2011", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1386.61", "stddev": "666.6", "min": "600.0", "max": "7423.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "October 2011"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "November 2011", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1399.46", "stddev": "704.7", "min": "609.0", "max": "7502.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "November 2011"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "December 2011", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1395.01", "stddev": "706.68", "min": "611.0", "max": "7540.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "December 2011"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "January 2012", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1398.58", "stddev": "709.84", "min": "613.0", "max": "7593.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "January 2012"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "February 2012", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1400.56", "stddev": "712.18", "min": "611.0", "max": "7639.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "February 2012"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "March 2012", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1403.4", "stddev": "713.28", "min": "607.0", "max": "7668.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "March 2012"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "April 2012", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1394.11", "stddev": "685.5", "min": "597.0", "max": "7675.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "April 2012"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "May 2012", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1405.6", "stddev": "718.11", "min": "590.0", "max": "7659.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "May 2012"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "June 2012", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1410.6", "stddev": "722.54", "min": "586.0", "max": "7665.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "June 2012"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "July 2012", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1411.4", "stddev": "724.08", "min": "587.0", "max": "7661.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "July 2012"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "August 2012", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1413.04", "stddev": "723.41", "min": "592.0", "max": "7657.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "August 2012"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "September 2012", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1398.92", "stddev": "690.44", "min": "596.0", "max": "7642.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "September 2012"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "October 2012", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1414.12", "stddev": "722.23", "min": "596.0", "max": "7656.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "October 2012"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "November 2012", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1414.4", "stddev": "724.35", "min": "592.0", "max": "7701.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "November 2012"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "December 2012", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1417.08", "stddev": "724.26", "min": "583.0", "max": "7743.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "December 2012"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "January 2013", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1417.8", "stddev": "727.82", "min": "581.0", "max": "7758.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "January 2013"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "February 2013", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1423.22", "stddev": "733.04", "min": "584.0", "max": "7755.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "February 2013"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "March 2013", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1426.83", "stddev": "737.74", "min": "592.0", "max": "7740.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "March 2013"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "April 2013", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1427.71", "stddev": "743.11", "min": "602.0", "max": "7834.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "April 2013"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "May 2013", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1417.89", "stddev": "712.97", "min": "617.0", "max": "8002.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "May 2013"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "June 2013", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1429.03", "stddev": "748.57", "min": "631.0", "max": "8184.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "June 2013"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "July 2013", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1424.55", "stddev": "718.89", "min": "639.0", "max": "8242.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "July 2013"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "August 2013", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1441.75", "stddev": "752.37", "min": "641.0", "max": "8191.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "August 2013"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "September 2013", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1445.9", "stddev": "752.97", "min": "636.0", "max": "8092.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "September 2013"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "October 2013", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1449.25", "stddev": "754.37", "min": "631.0", "max": "8019.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "October 2013"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "November 2013", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1453.74", "stddev": "756.71", "min": "630.0", "max": "8011.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "November 2013"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "December 2013", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1462.45", "stddev": "758.68", "min": "629.0", "max": "8017.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "December 2013"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "January 2014", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1463.08", "stddev": "761.39", "min": "630.0", "max": "8040.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "January 2014"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "February 2014", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1457.69", "stddev": "765.31", "min": "635.0", "max": "8059.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "February 2014"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "March 2014", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1460.21", "stddev": "769.31", "min": "634.0", "max": "8072.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "March 2014"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "April 2014", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1459.32", "stddev": "772.84", "min": "635.0", "max": "8024.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "April 2014"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "May 2014", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1443.29", "stddev": "740.44", "min": "642.0", "max": "7948.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "May 2014"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "June 2014", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1451.49", "stddev": "783.05", "min": "650.0", "max": "7855.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "June 2014"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "July 2014", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1455.27", "stddev": "791.84", "min": "649.0", "max": "8048.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "July 2014"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "August 2014", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1454.68", "stddev": "762.93", "min": "632.0", "max": "8331.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "August 2014"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "September 2014", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1468.53", "stddev": "773.95", "min": "621.0", "max": "8551.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "September 2014"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "October 2014", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1490.54", "stddev": "824.0", "min": "607.0", "max": "8717.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "October 2014"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "November 2014", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1505.79", "stddev": "833.28", "min": "605.0", "max": "8867.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "November 2014"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "December 2014", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1506.64", "stddev": "837.48", "min": "612.0", "max": "8953.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "December 2014"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "January 2015", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1510.54", "stddev": "841.02", "min": "625.0", "max": "9020.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "January 2015"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "February 2015", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1517.34", "stddev": "847.84", "min": "623.0", "max": "9112.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "February 2015"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "March 2015", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1523.18", "stddev": "855.42", "min": "617.0", "max": "9224.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "March 2015"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "April 2015", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1525.08", "stddev": "862.08", "min": "640.0", "max": "9317.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "April 2015"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "May 2015", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1536.6", "stddev": "869.17", "min": "643.0", "max": "9356.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "May 2015"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "June 2015", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1530.04", "stddev": "831.56", "min": "638.0", "max": "9369.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "June 2015"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "July 2015", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1548.32", "stddev": "879.07", "min": "634.0", "max": "9435.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "July 2015"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "August 2015", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1556.77", "stddev": "886.0", "min": "636.0", "max": "9536.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "August 2015"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "September 2015", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1552.94", "stddev": "890.43", "min": "642.0", "max": "9677.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "September 2015"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "October 2015", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1554.82", "stddev": "897.02", "min": "649.0", "max": "9820.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "October 2015"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "November 2015", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1550.24", "stddev": "902.55", "min": "641.0", "max": "9974.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "November 2015"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "December 2015", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1553.63", "stddev": "907.51", "min": "612.0", "max": "10088.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "December 2015"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "January 2016", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1554.45", "stddev": "909.38", "min": "612.0", "max": "10217.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "January 2016"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "February 2016", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1563.81", "stddev": "909.45", "min": "610.0", "max": "10159.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "February 2016"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "March 2016", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1554.87", "stddev": "864.98", "min": "607.0", "max": "10116.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "March 2016"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "April 2016", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1571.18", "stddev": "904.2", "min": "633.0", "max": "9978.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "April 2016"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "May 2016", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1566.0", "stddev": "863.88", "min": "650.0", "max": "9968.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "May 2016"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "June 2016", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1579.62", "stddev": "904.93", "min": "646.0", "max": "9868.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "June 2016"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "July 2016", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1578.5", "stddev": "904.77", "min": "644.0", "max": "9743.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "July 2016"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "August 2016", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1579.87", "stddev": "903.65", "min": "641.0", "max": "9565.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "August 2016"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "September 2016", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1577.06", "stddev": "900.67", "min": "641.0", "max": "9409.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "September 2016"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "October 2016", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1572.57", "stddev": "896.69", "min": "619.0", "max": "9293.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "October 2016"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "November 2016", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1574.54", "stddev": "896.03", "min": "623.0", "max": "9207.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "November 2016"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "December 2016", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1564.33", "stddev": "855.38", "min": "637.0", "max": "9106.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "December 2016"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "January 2017", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "1580.26", "stddev": "896.46", "min": "640.0", "max": "8983.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "January 2017"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "Population Rank_y", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "5794.52", "stddev": "3428.85", "min": "8.0", "max": "11885.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "Population Rank_y"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "November 2010ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.88", "stddev": "0.3", "min": "0.47200000000000003", "max": "2.742", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "November 2010ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "December 2010ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.89", "stddev": "0.3", "min": "0.47", "max": "2.728", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "December 2010ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "January 2011ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.89", "stddev": "0.3", "min": "0.47", "max": "2.616", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "January 2011ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "February 2011ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.89", "stddev": "0.29", "min": "0.47", "max": "2.5140000000000002", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "February 2011ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "March 2011ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.89", "stddev": "0.29", "min": "0.47", "max": "2.53", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "March 2011ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "April 2011ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.89", "stddev": "0.29", "min": "0.466", "max": "2.532", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "April 2011ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "May 2011ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.89", "stddev": "0.29", "min": "0.46399999999999997", "max": "2.5", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "May 2011ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "June 2011ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.89", "stddev": "0.29", "min": "0.466", "max": "2.48", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "June 2011ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "July 2011ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.88", "stddev": "0.28", "min": "0.47600000000000003", "max": "2.45", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "July 2011ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "August 2011ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.88", "stddev": "0.28", "min": "0.488", "max": "2.454", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "August 2011ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "September 2011ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.88", "stddev": "0.29", "min": "0.484", "max": "2.4659999999999997", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "September 2011ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "October 2011ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.88", "stddev": "0.29", "min": "0.47600000000000003", "max": "2.488", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "October 2011ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "November 2011ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.88", "stddev": "0.29", "min": "0.46399999999999997", "max": "2.5180000000000002", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "November 2011ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "December 2011ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.88", "stddev": "0.29", "min": "0.462", "max": "2.546", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "December 2011ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "January 2012ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.88", "stddev": "0.29", "min": "0.47", "max": "2.6", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "January 2012ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "February 2012ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.88", "stddev": "0.29", "min": "0.474", "max": "2.6260000000000003", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "February 2012ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "March 2012ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.88", "stddev": "0.29", "min": "0.478", "max": "2.638", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "March 2012ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "April 2012ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.89", "stddev": "0.3", "min": "0.478", "max": "2.6519999999999997", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "April 2012ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "May 2012ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.89", "stddev": "0.3", "min": "0.478", "max": "2.674", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "May 2012ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "June 2012ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.89", "stddev": "0.3", "min": "0.478", "max": "2.71", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "June 2012ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "July 2012ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.89", "stddev": "0.3", "min": "0.47600000000000003", "max": "2.736", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "July 2012ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "August 2012ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.89", "stddev": "0.3", "min": "0.46", "max": "2.7539999999999996", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "August 2012ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "September 2012ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.89", "stddev": "0.3", "min": "0.456", "max": "2.77", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "September 2012ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "October 2012ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.89", "stddev": "0.3", "min": "0.466", "max": "2.782", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "October 2012ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "November 2012ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.89", "stddev": "0.3", "min": "0.47200000000000003", "max": "2.792", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "November 2012ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "December 2012ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.89", "stddev": "0.3", "min": "0.466", "max": "2.8080000000000003", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "December 2012ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "January 2013ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.89", "stddev": "0.3", "min": "0.466", "max": "2.832", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "January 2013ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "February 2013ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.9", "stddev": "0.31", "min": "0.47", "max": "2.862", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "February 2013ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "March 2013ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.9", "stddev": "0.31", "min": "0.47600000000000003", "max": "2.89", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "March 2013ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "April 2013ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.9", "stddev": "0.31", "min": "0.474", "max": "2.8739999999999997", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "April 2013ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "May 2013ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.9", "stddev": "0.31", "min": "0.47200000000000003", "max": "2.8360000000000003", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "May 2013ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "June 2013ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.9", "stddev": "0.31", "min": "0.47600000000000003", "max": "2.784", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "June 2013ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "July 2013ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.9", "stddev": "0.31", "min": "0.47200000000000003", "max": "2.748", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "July 2013ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "August 2013ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.91", "stddev": "0.31", "min": "0.466", "max": "2.716", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "August 2013ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "September 2013ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.91", "stddev": "0.31", "min": "0.47", "max": "2.7", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "September 2013ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "October 2013ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.91", "stddev": "0.31", "min": "0.466", "max": "2.7", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "October 2013ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "November 2013ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.92", "stddev": "0.32", "min": "0.46399999999999997", "max": "2.7060000000000004", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "November 2013ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "December 2013ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.92", "stddev": "0.31", "min": "0.456", "max": "2.7", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "December 2013ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "January 2014ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.92", "stddev": "0.32", "min": "0.45", "max": "2.676", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "January 2014ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "February 2014ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.92", "stddev": "0.32", "min": "0.45", "max": "2.6060000000000003", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "February 2014ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "March 2014ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.92", "stddev": "0.32", "min": "0.456", "max": "2.5860000000000003", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "March 2014ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "April 2014ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.92", "stddev": "0.32", "min": "0.466", "max": "2.588", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "April 2014ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "May 2014ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.91", "stddev": "0.32", "min": "0.47200000000000003", "max": "2.636", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "May 2014ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "June 2014ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.91", "stddev": "0.33", "min": "0.46799999999999997", "max": "2.6519999999999997", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "June 2014ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "July 2014ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.92", "stddev": "0.33", "min": "0.462", "max": "2.676", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "July 2014ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "August 2014ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.92", "stddev": "0.33", "min": "0.456", "max": "2.7119999999999997", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "August 2014ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "September 2014ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.93", "stddev": "0.34", "min": "0.452", "max": "2.7460000000000004", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "September 2014ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "October 2014ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.94", "stddev": "0.34", "min": "0.444", "max": "2.804", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "October 2014ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "November 2014ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.94", "stddev": "0.35", "min": "0.44", "max": "2.85", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "November 2014ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "December 2014ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.95", "stddev": "0.35", "min": "0.442", "max": "2.912", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "December 2014ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "January 2015ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.95", "stddev": "0.35", "min": "0.446", "max": "2.966", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "January 2015ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "February 2015ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.95", "stddev": "0.36", "min": "0.446", "max": "3.0180000000000002", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "February 2015ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "March 2015ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.95", "stddev": "0.35", "min": "0.452", "max": "3.05", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "March 2015ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "April 2015ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.95", "stddev": "0.36", "min": "0.446", "max": "3.0580000000000003", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "April 2015ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "May 2015ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.96", "stddev": "0.36", "min": "0.43", "max": "3.036", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "May 2015ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "June 2015ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.96", "stddev": "0.37", "min": "0.434", "max": "2.9939999999999998", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "June 2015ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "July 2015ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.97", "stddev": "0.37", "min": "0.452", "max": "2.98", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "July 2015ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "August 2015ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.97", "stddev": "0.37", "min": "0.47", "max": "2.9760000000000004", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "August 2015ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "September 2015ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.97", "stddev": "0.37", "min": "0.47", "max": "2.978", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "September 2015ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "October 2015ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.97", "stddev": "0.37", "min": "0.456", "max": "2.966", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "October 2015ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "November 2015ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.97", "stddev": "0.37", "min": "0.434", "max": "2.938", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "November 2015ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "December 2015ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.97", "stddev": "0.37", "min": "0.41600000000000004", "max": "2.938", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "December 2015ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "January 2016ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.97", "stddev": "0.37", "min": "0.418", "max": "2.97", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "January 2016ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "February 2016ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.98", "stddev": "0.38", "min": "0.43", "max": "2.9939999999999998", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "February 2016ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "March 2016ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.98", "stddev": "0.38", "min": "0.442", "max": "3.0239999999999996", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "March 2016ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "April 2016ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.99", "stddev": "0.38", "min": "0.462", "max": "3.042", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "April 2016ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "May 2016ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.99", "stddev": "0.38", "min": "0.48200000000000004", "max": "3.0580000000000003", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "May 2016ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "June 2016ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.99", "stddev": "0.39", "min": "0.484", "max": "3.076", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "June 2016ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "July 2016ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.99", "stddev": "0.38", "min": "0.48200000000000004", "max": "3.088", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "July 2016ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "August 2016ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.99", "stddev": "0.39", "min": "0.47600000000000003", "max": "3.096", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "August 2016ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "September 2016ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.99", "stddev": "0.39", "min": "0.47600000000000003", "max": "3.102", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "September 2016ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "October 2016ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.99", "stddev": "0.39", "min": "0.466", "max": "3.094", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "October 2016ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "November 2016ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.99", "stddev": "0.39", "min": "0.47", "max": "3.0580000000000003", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "November 2016ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "December 2016ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.99", "stddev": "0.39", "min": "0.49", "max": "3.0180000000000002", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "December 2016ppsf"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "January 2017ppsf", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "0.99", "stddev": "0.39", "min": "0.49", "max": "3.0180000000000002", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "January 2017ppsf"}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run zillowregressionHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionRegression(autofe, ["City Code", "Population Rank_x", "November 2010", "December 2010", "January 2011", "February 2011", "March 2011", "April 2011", "May 2011", "June 2011", "July 2011", "August 2011", "September 2011", "October 2011", "November 2011", "December 2011", "January 2012", "February 2012", "March 2012", "April 2012", "May 2012", "June 2012", "July 2012", "August 2012", "September 2012", "October 2012", "November 2012", "December 2012", "January 2013", "February 2013", "March 2013", "April 2013", "May 2013", "June 2013", "July 2013", "August 2013", "September 2013", "October 2013", "November 2013", "December 2013", "January 2014", "February 2014", "March 2014", "April 2014", "May 2014", "June 2014", "July 2014", "August 2014", "September 2014", "October 2014", "November 2014", "December 2014", "January 2015", "February 2015", "March 2015", "April 2015", "May 2015", "June 2015", "July 2015", "August 2015", "September 2015", "October 2015", "November 2015", "December 2015", "January 2016", "February 2016", "March 2016", "April 2016", "May 2016", "June 2016", "July 2016", "August 2016", "September 2016", "October 2016", "November 2016", "December 2016", "Population Rank_y", "November 2010ppsf", "December 2010ppsf", "January 2011ppsf", "February 2011ppsf", "March 2011ppsf", "April 2011ppsf", "May 2011ppsf", "June 2011ppsf", "July 2011ppsf", "August 2011ppsf", "September 2011ppsf", "October 2011ppsf", "November 2011ppsf", "December 2011ppsf", "January 2012ppsf", "February 2012ppsf", "March 2012ppsf", "April 2012ppsf", "May 2012ppsf", "June 2012ppsf", "July 2012ppsf", "August 2012ppsf", "September 2012ppsf", "October 2012ppsf", "November 2012ppsf", "December 2012ppsf", "January 2013ppsf", "February 2013ppsf", "March 2013ppsf", "April 2013ppsf", "May 2013ppsf", "June 2013ppsf", "July 2013ppsf", "August 2013ppsf", "September 2013ppsf", "October 2013ppsf", "November 2013ppsf", "December 2013ppsf", "January 2014ppsf", "February 2014ppsf", "March 2014ppsf", "April 2014ppsf", "May 2014ppsf", "June 2014ppsf", "July 2014ppsf", "August 2014ppsf", "September 2014ppsf", "October 2014ppsf", "November 2014ppsf", "December 2014ppsf", "January 2015ppsf", "February 2015ppsf", "March 2015ppsf", "April 2015ppsf", "May 2015ppsf", "June 2015ppsf", "July 2015ppsf", "August 2015ppsf", "September 2015ppsf", "October 2015ppsf", "November 2015ppsf", "December 2015ppsf", "January 2016ppsf", "February 2016ppsf", "March 2016ppsf", "April 2016ppsf", "May 2016ppsf", "June 2016ppsf", "July 2016ppsf", "August 2016ppsf", "September 2016ppsf", "October 2016ppsf", "November 2016ppsf", "December 2016ppsf", "January 2017ppsf", "City_stringindexer", "Metro_stringindexer", "County_stringindexer", "State_stringindexer"], "January 2017")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML ['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    df = df[columnNames]
    R2 = np.round(sklearn.metrics.r2_score(y_test, y_predicted), 1)
    Mean_Squared_Error = np.round(sklearn.metrics.mean_squared_error(y_test, y_predicted), 1)
    Mean_Absolute_Error = np.round(sklearn.metrics.mean_absolute_error(y_test, y_predicted), 1)
    display(" R2 score of Prediction on test data    : %s"%R2)
    display(" Mean Squared Error of Prediction on test data    : %s"%Mean_Squared_Error)
    display(" Mean Absolute Error of Prediction on test data   : %s"%Mean_Absolute_Error)
    display(df.head())
except Exception as ex:
    logging.error(ex)

